<a href="https://colab.research.google.com/github/younesabdolmalaky/Pistachio/blob/main/catboost28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install imblearn
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.3 MB/s eta 0:00:00


In [3]:
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

In [4]:
with open('/content/drive/MyDrive/Pistachio/vec/train.pickle', 'rb') as f:
    train = pickle.load(f)

with open('/content/drive/MyDrive/Pistachio/vec/test.pickle', 'rb') as f:
    test = pickle.load(f)

X_train = train[:, 20:48].astype(np.float32)
y_train = train[:, 49].astype(np.float32)

X_test = test[:, 20:48].astype(np.float32)
y_test = test[:, 49].astype(np.float32)

In [5]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [14]:
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=3,
    random_seed=42,
    loss_function='Logloss',
    eval_metric='F1',
    verbose=100
)

In [15]:
model.fit(
    X_train_resampled, y_train_resampled,
    eval_set=(X_test, y_test),
    early_stopping_rounds=50,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7065990	test: 0.6575342	best: 0.6575342 (0)	total: 5.47ms	remaining: 5.46s
100:	learn: 0.9979675	test: 0.9757412	best: 0.9757412 (100)	total: 539ms	remaining: 4.8s
200:	learn: 1.0000000	test: 0.9890710	best: 0.9890710 (159)	total: 1.08s	remaining: 4.29s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9917808219
bestIteration = 202

Shrink model to first 203 iterations.


In [17]:
y_pred = model.predict(X_train_resampled)
print(classification_report(y_pred, y_train_resampled))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       985
         1.0       1.00      1.00      1.00       985

    accuracy                           1.00      1970
   macro avg       1.00      1.00      1.00      1970
weighted avg       1.00      1.00      1.00      1970



In [16]:
y_pred = model.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99       250
         1.0       0.98      1.00      0.99       181

    accuracy                           0.99       431
   macro avg       0.99      0.99      0.99       431
weighted avg       0.99      0.99      0.99       431

